In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
import polars as pl 
from pathlib import Path
from datetime import datetime, timedelta

from sklearn.metrics import mean_absolute_error

from notebook_utils import insert_parent_in_path
insert_parent_in_path()

from src.plots import plot_ts
from src.paths import *
from src.config import TODAY_IS, ModelConfig
from src.dwh import run_database_operation
from src.train import split_into_train_and_test

# Load from DWH

In [4]:
df = run_database_operation(
    operation="fetch_pickup_data",
    from_date=TODAY_IS - timedelta(days=max(ModelConfig.LAGS)+1),
    to_date=TODAY_IS,
    pickup_locations=[43]
)

In [5]:
import joblib
from src.paths import MODEL_DIR
model = joblib.load(MODEL_DIR / "baseline_model.pkl")



AttributeError: Can't get attribute 'LagTransformer' on <module 'src.pipeline' from 'c:\\Users\\selaf\\OneDrive\\Documentos\\dev\\real-ml-course\\taxi_demand_predictor\\src\\pipeline.py'>

In [21]:
predictions = model.predict(df)
data_with_predictions = df.join(predictions, on=ModelConfig.TS_INDEX, how="inner")
plot_ts(data_with_predictions, ["num_pickup", "prediction"])

In [22]:
data_with_predictions

pickup_datetime_hour,pickup_location_id,num_pickup,prediction
date,i16,f64,f64
2023-05-31,43,1786.0,1788.25


## Muti-step forecast